In [4]:
import sys
sys.path.append('../input/iterative-stratification/iterative-stratification-master')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [5]:
import warnings
warnings.filterwarnings("ignore")

In [11]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ReduceLROnPlateau, ModelCheckpoint
import tensorflow_addons as tfa

from sklearn.metrics import log_loss
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

TypeError: Descriptors cannot be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [ ]:
# Seed Everythig !!
def seed_everything(seed=2020): 
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

In [ ]:
SEED = 42
NFOLDS = 5
EPOCHS = 50
BATCH_SIZE = 128
ID = 'sig_id'
seed_everything(SEED)

In [ ]:
train = pd.read_csv('train_features.csv')
target = pd.read_csv('train_targets_scored.csv')
test = pd.read_csv('test_features.csv')

In [ ]:
train.sample(5)

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
12981,id_8baf3472c,trt_cp,48,D2,-0.3555,0.3902,0.0050,0.0950,0.1351,-0.2835,...,0.5282,0.1966,0.5799,0.7647,0.2427,-0.1676,-0.3130,-0.2610,0.6027,0.4509
4290,id_2e0ee5f8c,trt_cp,72,D2,-0.2093,-0.3729,-0.6765,0.0670,0.0843,0.5460,...,-0.4870,-0.7246,0.3958,-0.3892,-0.8434,0.3928,0.1331,-0.5221,-0.0787,-0.8207
11666,id_7dc9cda65,trt_cp,72,D2,0.3015,-0.4663,0.1393,-0.0744,-0.4217,-0.6363,...,0.3933,0.2525,-0.5392,0.1192,0.3145,0.7621,-1.1720,0.2109,-0.0866,0.1781
1056,id_0ac0decf0,trt_cp,72,D2,-0.0249,-0.3802,-0.1626,-0.0891,0.2932,0.2894,...,0.5833,0.9230,0.0516,-0.5003,0.8374,-0.7396,0.7603,-0.0362,0.5500,0.9635
423,id_047715270,trt_cp,72,D1,5.8740,-0.4165,0.4836,-1.2200,1.1140,-3.8620,...,-9.9530,-9.9530,-9.8580,-9.9530,-9.9530,-8.7210,-9.9530,-8.3560,-8.7480,-8.3770


In [ ]:
target.sample(5)

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
15699,id_a8d1bf1e6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3934,id_2a1c6853a,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17162,id_b8ce0ee3f,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10586,id_71f9139b6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5212,id_38263582c,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
test.sample(5)

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
2474,id_9e634c4ab,trt_cp,24,D1,-0.7487,0.2676,1.5970,0.7490,-0.1576,1.9450,...,-0.0217,-0.2352,0.4443,-1.2860,0.4270,0.2349,0.0351,-0.3772,-0.5158,-0.2441
191,id_0c9f1581e,trt_cp,24,D1,-1.3340,-0.2648,0.9757,0.2981,0.3699,0.1025,...,0.5917,0.7217,0.5966,0.0245,0.1777,-0.1040,1.0220,1.0610,0.2126,-0.7211
3748,id_f10547735,ctl_vehicle,24,D2,0.0808,-0.9983,0.3287,-0.8688,-1.1010,-1.3600,...,-1.0050,0.4401,-0.2798,0.2143,0.5364,0.4076,-0.6087,-0.3178,0.5843,0.4781
486,id_20299e6ec,trt_cp,72,D1,-0.4647,0.6066,0.9777,-0.1882,-0.7054,0.1581,...,0.6347,1.1180,0.6250,0.1235,0.9126,1.1890,-0.2782,0.0668,1.0080,-0.0882
2875,id_b9598d48a,trt_cp,48,D2,-0.2146,0.5224,0.5711,0.7461,-0.5208,-1.1310,...,-0.1954,-0.8403,-0.0662,0.1985,-0.5045,-1.2340,-0.4198,-0.9315,-0.4884,-0.6673


In [ ]:
sub_df = pd.read_csv('sample_submission.csv')
sub_df.sample(5)

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
460,id_1e82d97d1,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
2817,id_b4c7e09c9,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
2329,id_956df9791,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
2769,id_b204db0a9,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
2105,id_876ece275,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5


In [ ]:
top_feats = [  1,   2,   3,   4,   5,   6,   7,   9,  11,  14,  15,  16,  17,
        18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  29,  30,  31,
        32,  33,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  46,
        47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  58,  59,  60,
        61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,
        74,  75,  76,  78,  79,  80,  81,  82,  83,  84,  86,  87,  88,
        89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101,
       102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114,
       115, 116, 117, 118, 120, 121, 122, 123, 124, 125, 126, 127, 128,
       129, 130, 131, 132, 133, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 149, 150, 151, 152, 153, 154, 155, 156, 157,
       158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170,
       171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183,
       184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 197,
       198, 199, 200, 202, 203, 204, 205, 206, 208, 209, 210, 211, 212,
       213, 214, 215, 216, 217, 218, 219, 220, 221, 223, 224, 225, 226,
       227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239,
       240, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253,
       254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266,
       267, 268, 269, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280,
       281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 294,
       295, 296, 298, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309,
       310, 311, 312, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323,
       324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336,
       337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349,
       350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362,
       363, 364, 365, 366, 367, 368, 369, 370, 371, 374, 375, 376, 377,
       378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 390, 391,
       392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404,
       405, 406, 407, 408, 409, 411, 412, 413, 414, 415, 416, 417, 418,
       419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431,
       432, 434, 435, 436, 437, 438, 439, 440, 442, 443, 444, 445, 446,
       447, 448, 449, 450, 453, 454, 456, 457, 458, 459, 460, 461, 462,
       463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475,
       476, 477, 478, 479, 481, 482, 483, 484, 485, 486, 487, 488, 489,
       490, 491, 492, 493, 494, 495, 496, 498, 500, 501, 502, 503, 505,
       506, 507, 509, 510, 511, 512, 513, 514, 515, 518, 519, 520, 521,
       522, 523, 524, 525, 526, 527, 528, 530, 531, 532, 534, 535, 536,
       538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 549, 550, 551,
       552, 554, 557, 559, 560, 561, 562, 565, 566, 567, 568, 569, 570,
       571, 572, 573, 574, 575, 577, 578, 580, 581, 582, 583, 584, 585,
       586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 599,
       600, 601, 602, 606, 607, 608, 609, 611, 612, 613, 615, 616, 617,
       618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630,
       631, 632, 633, 634, 635, 636, 637, 638, 639, 641, 642, 643, 644,
       645, 646, 647, 648, 649, 650, 651, 652, 654, 655, 656, 658, 659,
       660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670, 671, 672,
       673, 674, 675, 676, 677, 678, 679, 680, 681, 682, 683, 684, 685,
       686, 687, 688, 689, 691, 692, 693, 694, 695, 696, 697, 699, 700,
       701, 702, 704, 705, 707, 708, 709, 710, 711, 713, 714, 716, 717,
       718, 720, 721, 723, 724, 725, 726, 727, 728, 729, 730, 731, 732,
       733, 734, 735, 737, 738, 739, 740, 742, 743, 744, 745, 746, 747,
       748, 749, 750, 751, 752, 753, 754, 755, 756, 757, 759, 760, 761,
       762, 763, 764, 765, 766, 767, 768, 769, 770, 771, 772, 773, 774,
       775, 776, 777, 779, 780, 781, 782, 783, 784, 785, 786, 787, 788,
       789, 790, 792, 793, 794, 795, 796, 797, 798, 800, 801, 802, 803,
       804, 805, 806, 808, 809, 811, 813, 814, 815, 816, 817, 818, 819,
       821, 822, 823, 825, 826, 827, 828, 829, 830, 831, 832, 834, 835,
       837, 838, 839, 840, 841, 842, 845, 846, 847, 848, 850, 851, 852,
       854, 855, 856, 858, 859, 860, 861, 862, 864, 866, 867, 868, 869,
       870, 871, 872, 873, 874]
print(len(top_feats))

785


In [ ]:
# keep the ID column separate
train_id = train[ID]
test_id = test[ID]
sub_id = sub_df[ID]

In [ ]:
#Keeping the important features only
important_cols = []
train_cols = train.columns
for i in range(len(train_cols)):
    if i in top_feats:
        important_cols.append(train_cols[i])
print(len(important_cols))

train = train[important_cols]
test = test[important_cols]

785


In [ ]:
def preprocess(df):
    _df = df.copy()
    _df['cp_type'] = _df['cp_type'].apply(lambda x : 1 if x == 'ctl_vehicle' else 0)
    _df['cp_dose'] = _df['cp_dose'].apply(lambda x : 1 if x == 'D2' else 0)
    return _df

train = preprocess(train)
test = preprocess(test)

del target[ID]

target = target.loc[train['cp_type']==0].reset_index(drop=True)
train = train.loc[train['cp_type']==0].reset_index(drop=True)

In [ ]:
# ===== SCALING ===== #

train['WHERE'] = 'train'
test['WHERE'] = 'test'
temp = pd.concat([train, test], ignore_index=True)

scaler = StandardScaler()
temp.iloc[:, :-1] = scaler.fit_transform(temp.iloc[:, :-1])

In [ ]:
temp.sample(5)

,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-5,g-7,g-10,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,WHERE
5445,-0.11832,1.235906,1.020102,-0.622480,-0.166646,-0.629598,0.063807,-0.100989,-0.572457,0.043495,...,-0.171707,0.341263,0.428503,0.328154,0.332045,0.105462,0.566788,-0.154404,-0.082743,train
7739,-0.11832,1.235906,1.020102,-0.604301,0.168752,-0.208777,-0.572117,1.144524,1.207189,0.168143,...,-0.087252,0.401075,0.633011,0.332770,0.437271,0.035787,0.946463,0.040944,0.457553,train
2245,-0.11832,-0.000906,1.020102,-0.283711,-1.917976,-0.062321,0.512589,0.375675,0.318163,0.188270,...,0.516042,0.071966,0.589857,0.329123,0.630064,0.512507,0.365090,0.139164,0.330961,train
25286,-0.11832,-1.237719,-0.980294,-0.354945,0.199186,2.025160,0.292756,-0.298279,0.069037,0.423386,...,0.510937,-0.013834,0.106479,0.792188,0.668948,-0.378495,0.023377,0.548605,0.187938,test
24757,-0.11832,-1.237719,-0.980294,2.025457,-0.310085,-0.780667,-0.045796,-1.341614,0.762676,0.638299,...,0.728420,0.363860,0.200492,0.596040,0.219187,0.024124,0.246391,0.577588,0.332455,test


In [ ]:
# separate train and test data
train = temp.loc[temp.WHERE == 'train']
test = temp.loc[temp.WHERE == 'test']
del train['WHERE']
del test['WHERE']

In [ ]:
# Define callbacks

def get_early_stopper():
    earlyStop = EarlyStopping( monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto',
        baseline=None, restore_best_weights=True)
    return earlyStop


def get_lr_callback(batch_size = 64, plot = False):
    """Returns a lr_scheduler callback which is used for training.
    Feel free to change the values below!
    """
    lr_start   = 0.001
    lr_max     = 0.001 * BATCH_SIZE # higher batch size --> higher lr
    lr_min     = 0.00001
    # 30% of all epochs are used for ramping up the LR and then declining starts
    lr_ramp_ep = EPOCHS * 0.3
    lr_sus_ep  = 0
    lr_decay   = 0.9

    def lr_scheduler(epoch):
            if epoch < lr_ramp_ep:
                lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start

            elif epoch < lr_ramp_ep + lr_sus_ep:
                lr = lr_max

            else:
                lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min

            return lr
    
    if plot == False:
        # get the Keras-required callback with our LR for training
        lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_scheduler,verbose = 2)
        return lr_callback 
    
    else: 
        return lr_scheduler

    
def reduce_lr_on_plateau():
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3,
                                  epsilon = 1e-4, mode = 'min', verbose=1)
    return reduce_lr

In [ ]:
def make_model(l): 
    model = tf.keras.Sequential([
        Input(l),
        BatchNormalization(),
        Dropout(0.2),
        tfa.layers.WeightNormalization(Dense(2048, activation="elu")),
        BatchNormalization(),
        Dropout(0.5),
        tfa.layers.WeightNormalization(Dense(1024, activation="elu")),
        BatchNormalization(),
        Dropout(0.5),
        tfa.layers.WeightNormalization(Dense(206, activation="sigmoid")),
    ])

    model.compile(loss='binary_crossentropy',
                 #optimizer = tf.keras.optimizers.Adam(),
                 #optimizer = tfa.optimizers.LazyAdam(0.001),
                 optimizer = tfa.optimizers.Lookahead(tf.optimizers.Adam(), sync_period=10),
                 #optimizer = tf.keras.optimizers.SGD(),
                  metrics=["accuracy"]
                 )
    return model

In [ ]:
net = make_model(len(train.columns))
net.summary()

TypeError: optimizer is not an object of tf.keras.optimizers.legacy.Optimizer 

In [ ]:
print(tf.__version__)

2.14.0


In [ ]:
print(tfa.__version__)

0.22.0
